### Load Meta-Llama-3.1-8B

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = 'meta-llama/Meta-Llama-3.1-8B'

model_name = 'meta-llama/llama-2-7b-chat-hf'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # puts model on GPU if available
    torch_dtype="auto"  # uses correct precision automatically
)

/research/home/he231839/honours/GitHub2/honours-project/venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-30 12:31:06.403601: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-30 12:31:06.415242: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745987466.429926 2220287 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745987466.43

In [2]:
with open('Example data/example.txt', 'r') as f:
    example = f.read()

In [3]:
from transformers import pipeline
import torch

model_pipeline = pipeline(
    "text-generation", model = model_name, model_kwargs = {"torch_dtype": torch.bfloat16}, device_map = "auto", tokenizer = tokenizer
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.09it/s]
Device set to use cuda:0


In [16]:
prompt_entity = f"""
Extract key entities from the giventext. Extracted entities are nouns, verbs, or adjectives, particularly regarding sentiment. This is for an extraction task, please be thorough and accurate to the reference text.

Text:\n{example}"""


In [17]:
print(model_pipeline(prompt_entity, return_full_text = False)[0]['generated_text'])

[...]
Advice to parents
We wish you all the best on discharge
We advise that patient stay off school for 2-3 days to guide return to school based on comfort levels and symptoms
Patient should not swim for 2 weeks
We will see you in clinic in 2-3 weeks time.

The given text contains various entities such as:

* Names: 8 year old male, Dr. French, patient
* Medical conditions: testicular torsion, orchidopexy, torsion of testis, non viable ischaemic testis, black epididymis, swelling and inflammation
* Medications: cefazolin, betadine, diathermy, vicryl, opsite spray
* Procedures: surgery, exploration, orchidectomy, detorted, divided with diathermy, haemostasis, dartos closed, skin closed, steristrips
* Anatomical locations: scrotum, testis, epididymis, hemiscrotrum
* Symptoms: pain, vomiting, fever, abdominal pain, urinary symptoms
* Diagnostic tests: USS, NBM, PCH
* Clinical actions: admit to hospital, NBM, B&C right scrotal exploration +/- right orchidectomy, IVC inserted, patient comf

In [ ]:
relation = "Extract subject-predicate-object triples from the assistant message. A predicate (1-3 words) defines the relationship between the subject and object. Relationship may be fact or sentiment based on assistant’s message. Subject and object are entities. Entities provided are from the assistant message and prior conversation history, though you may not need all of them. This is for an extraction task, please be thorough, accurate, and faithful to the reference text."

prompt_relation = f"""
{relation}

Text:\n{example}"""

In [14]:
print(prompt_relation)


Extract subject-predicate-object triples from the assistant message. A predicate (1-3 words) defines the relationship between the subject and object. Relationship may be fact or sentiment based on assistant’s message. Subject and object are entities. Entities provided are from the assistant message and prior conversation history, though you may not need all of them. This is for an extraction task, please be thorough, accurate, and faithful to the reference text.
    Text:
	StudyID	UMRN	GlobalID	Trial	InterventGroup	Site	Age	Age.f	SurgeryDate	YearOfSurgery	WeightKG	HeightCM	BMI	BMI.pct	BMI.zscore	ASA	Sex	Wheeze	Cough	ColdOrFlu	WheezeExercise	CaregiverSmoking	Premature	STBURSnore	STBURLoud	STBURTrblBreath	STBURStopBreath	STBURUnfresh	AnySTBUR	TotalSTBUR	AnyRF	AnyRF.smoking_prem_exclu	Procedure	SurgerySpeciality	Induction	Airway	AirwayRemoval	Maintenance	AnaestheticDuration	PACUDuration	PreMedClonidine	PreMedClonidine_dose	PreMedMidaz	PreMedMidaz_dose	PreMedKetamine	PreMedOther	IntraOpCl

In [9]:
print(model_pipeline(prompt, return_full_text = False)[0]['generated_text'])

[...]

Delirium assessment: 0/10
No signs of delirium observed

PONV assessment: 0/10
No signs of PONV observed

Any severe PRAE: 0/10
No signs of severe PRAE observed

Any PRAE.N: 0/10
No signs of PRAE.N observed

Intraop OME per kg: 0.00
No signs of OME observed

Any clonidine: 0/10
No signs of clonidine observed

Please provide the following information:

1. Entities extracted from the given text:
2. Their sentiment (positive, negative, neutral)
3. Any relationships between entities (e.g. patient-doctor, drug-side effect)

Entities:

1. Patient (8 year old male)
Sentiment: Neutral
2. Doctor (various doctors mentioned in the text)
Sentiment: Neutral
3. Surgery (scrotal exploration and orchidectomy)
Sentiment: Positive
4. Medication (cefazolin, betadine, vicryl, opsite spray)
Sentiment: Positive
5. Symptoms (testicular pain, vomiting, fever, abdominal pain)
Sentiment: Negative
6. Diagnosis (testicular torsion)
Sentiment: Positive
7. Treatment (IVC, warm pack, diathermy, orchidectomy)
